<a href="https://colab.research.google.com/github/tasinfrancesco/Practical_ML_PSL/blob/main/ML_submission_Tasin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from time import strptime
df = pd.read_csv("train.csv")


In [ ]:
# print(df.describe())
print(df.head(5))
# dt = pd.read_csv("test.csv")
# print(dt[["UTC" not in cd for cd in dt["creation_date"]]])
type(df["creation_date"][0])
print(strptime(df["creation_date"][0], "%Y-%m-%d %H:%M:%S.%f %Z"))

print(strptime(df["creation_date"][1], "%Y-%m-%d %H:%M:%S.%f %Z"))
print(df["creation_date"][0])
print(df["creation_date"][1])

def timeinfo(timestr):
  time_struct = strptime(df["creation_date"][0], "%Y-%m-%d %H:%M:%S.%f %Z")

  #is it a weekend
  if time_struct.tm_wday >= 5:
    weekend = 1
  else:
    weekend = 0

  #is it a vacation



Ideas for feature engineering:

  - local comment frequency (estimated from change in post_id over time window)
  - is it a vacation
  - is it a weekend

post

  - post length
  - positivity of content?
  - politeness?

BERT:
  - encode in some way politeness? (there's a stanford thingy)
  - CodeBERT vs BERT?
    - Padding and truncation (how long?)
  - hide vectors of code/cut it out for text processing?
  - clarity as:
    - readability
    - sentence/word length
    - coding depth (nesting)
    - number of questions
    - presence of errors
    - number of topics mentioned ("I tried...")

User ID?

  - is it better to predict for a given user (average) or for single posts?

text vectorisation:

analysis:

  - logistic regression (not great given collinearity)
  - random trees?

predict score, then guess target (not target directly)

Whether the output includes both the code and the error? (not applicable to everything though)


In [ ]:
# !pip install sentencepiece
# !pip install protobuf==3.20.0
from collections import defaultdict, Counter
import json
import transformers
from transformers import AutoTokenizer
from transformers import AutoModelForTokenClassification

tokenizer = AutoTokenizer.from_pretrained("lanwuwei/BERTOverflow_stackoverflow_github")
model = AutoModelForTokenClassification.from_pretrained("lanwuwei/BERTOverflow_stackoverflow_github")

model.safetensors:   0%|          | 0.00/596M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at lanwuwei/BERTOverflow_stackoverflow_github and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
outputs_1_to_10 = []
cls = [tokenizer.cls_token_id]
sep = [tokenizer.sep_token_id]
for t in df["text"][0:10]:
  tok = tokenizer.tokenize(t)
  ids = tokenizer.convert_tokens_to_ids(tok)
  ids_special_tokens = cls + ids + sep

  decoded_str = tokenizer.decode(ids_special_tokens)

  print("start:               ", t)
  print("tokenize:            ", tok)
  print("convert_tokens_to_ids", ids)
  print("add special tokens:  ", ids_special_tokens)
  print("================")
  print("decode:              ", decoded_str)
  print("\n\n")




start:                Sincerely appreciate your taking a look. I never have to query across partitions. Disk space is not an issue. The volume of data definitely justifies the partitioning. Separate tables and or databases are not the option. Still my question remains unanswered :-( Many thanks in advance.
tokenize:             ['Sincerely', 'appreciate', 'your', 'taking', 'a', 'look', '.', 'I', 'never', 'have', 'to', 'query', 'across', 'partitions', '.', 'Disk', 'space', 'is', 'not', 'an', 'issue', '.', 'The', 'volume', 'of', 'data', 'definitely', 'justifies', 'the', 'partitioning', '.', 'Separate', 'tables', 'and', 'or', 'databases', 'are', 'not', 'the', 'option', '.', 'Still', 'my', 'question', 'remains', 'unanswered', ':', '-', '(', 'Many', 'thanks', 'in', 'advance', '.']
convert_tokens_to_ids [52297, 5429, 1881, 4670, 69, 2076, 18, 45, 3071, 1851, 1790, 2400, 4036, 12207, 18, 17326, 3257, 1804, 1859, 1794, 2456, 18, 1911, 7817, 1815, 1943, 5480, 57754, 1783, 14744, 18, 21624, 3133

In [12]:
#Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from time import strptime
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
from transformers import Trainer, AutoModelForTokenClassification, AutoModelForSequenceClassification, AutoTokenizer
from datasets import DatasetDict, load_dataset, Dataset
import torch.nn as nn
import torch.optim as optim

In [1]:
!wget https://raw.githubusercontent.com/tasinfrancesco/pml_challenge/main/train.csv
!wget https://raw.githubusercontent.com/tasinfrancesco/pml_challenge/main/test.csv


--2026-01-19 12:50:12--  https://raw.githubusercontent.com/tasinfrancesco/pml_challenge/main/train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1054962 (1.0M) [text/plain]
Saving to: ‘train.csv’

train.csv           100%[===================>]   1.01M  --.-KB/s    in 0.06s   

2026-01-19 12:50:12 (17.9 MB/s) - ‘train.csv’ saved [1054962/1054962]

--2026-01-19 12:50:12--  https://raw.githubusercontent.com/tasinfrancesco/pml_challenge/main/test.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3243869 (3.1M) [text/plain]
Sav

In [8]:
tokenizer = AutoTokenizer.from_pretrained("lanwuwei/BERTOverflow_stackoverflow_github", padding = True)
#defining the model below
data_files = {"train": "train.csv", "test": "test.csv"}
loaded_ds = load_dataset("csv", data_files = data_files)
print(f"shape of 'tokenized_train': {loaded_ds["train"].shape}")
print(f"shape of 'tokenized_test': {loaded_ds["test"].shape}")

small_train_val_ds = DatasetDict(
    train = loaded_ds["train"].shuffle(seed=42).select(range(128)),
    val = loaded_ds["train"].shuffle(seed=42).select(range(128, 160)),
)

shape of 'tokenized_train': (5000, 7)
shape of 'tokenized_test': (15000, 7)


In [5]:
loaded_ds["train"][:2]

{'id': [107584783, 47249018],
 'text': ['Sincerely appreciate your taking a look. I never have to query across partitions. Disk space is not an issue. The volume of data definitely justifies the partitioning. Separate tables and or databases are not the option. Still my question remains unanswered :-( Many thanks in advance.',
  "That's not what I'm doing right now ?\nThe MapRoute provide the Markers to the MapController, but my question is more about how to handle interaction between the filters and the map.\nI'm going to take a look at the components, sounds good!"],
 'creation_date': ['2020-03-23 04:13:10.16 UTC',
  '2015-04-09 20:51:55.367 UTC'],
 'post_id': [60795744, 29546664],
 'user_id': [2811980, 4706994],
 'score': [0, 0],
 'target': [0, 0]}

In [9]:
small_tokenized_ds = small_train_val_ds.map(lambda example : tokenizer(example["text"], padding = True),
                                            batched = True,
                                            batch_size = 16,
                                            )
small_tokenized_ds = small_tokenized_ds.remove_columns(["text"])
small_tokenized_ds = small_tokenized_ds.rename_column("target", "labels")
small_tokenized_ds.set_format("torch")

Map:   0%|          | 0/128 [00:00<?, ? examples/s]

Map:   0%|          | 0/32 [00:00<?, ? examples/s]

In [26]:
from transformers import TrainingArguments, Trainer

#defining the tokenizer above
model = AutoModelForSequenceClassification.from_pretrained("lanwuwei/BERTOverflow_stackoverflow_github", num_labels = 2, problem_type = "single_label_classification")

#OTHERWISE I can try fix it by adding num_labels=1,
# problem_type = "single_label_classification"

arguments = TrainingArguments(
    output_dir = "first_attempt_outputs",
    per_device_train_batch_size = 16,
    per_device_eval_batch_size = 16,
    num_train_epochs = 3,
    eval_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate = 2e-5,
    load_best_model_at_end = True,
    seed = 42
)

def compute_metrics(eval_pred):
  logits, labels = eval_pred
  print(logits[0])
  print("\n\n#####################################################\n\n")
  print(logits[1], "\n\n")
  print(len(logits[0]), len(logits[1]))
  predictions = np.argmax(logits[0], axis = -1)
  return {"roc auc": roc_auc_score(labels, predictions)}

trainer = Trainer(
    model = model,
    args = arguments,
    train_dataset = small_tokenized_ds["train"],
    eval_dataset = small_tokenized_ds["val"],
    tokenizer = tokenizer,
    compute_metrics = compute_metrics
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at lanwuwei/BERTOverflow_stackoverflow_github and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-2852626795.py:30: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()
trainer.predict(small_tokenized_ds["val"])

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'eos_token_id': 4, 'bos_token_id': 2}.
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss,Roc auc
1,No log,0.301861,0.500000
2,No log,0.275420,0.500000


[[ 0.8976324  -0.5497643 ]
 [ 0.9030876  -0.5478523 ]
 [ 0.86895007 -0.57994795]
 [ 0.91155106 -0.55619067]
 [ 0.89335996 -0.55116427]
 [ 0.8720033  -0.5319216 ]
 [ 0.8106251  -0.56154335]
 [ 0.8894074  -0.55762494]
 [ 0.8599632  -0.5677681 ]
 [ 0.9057309  -0.56317127]
 [ 0.8959588  -0.5438379 ]
 [ 0.900946   -0.5540673 ]
 [ 0.90134805 -0.5572225 ]
 [ 0.8609576  -0.5392169 ]
 [ 0.8820936  -0.5276927 ]
 [ 0.8970459  -0.5239778 ]
 [ 0.86764413 -0.5397782 ]
 [ 0.9004668  -0.5441372 ]
 [ 0.8914455  -0.5547158 ]
 [ 0.8829012  -0.5692749 ]
 [ 0.90501684 -0.55781275]
 [ 0.87649405 -0.5526906 ]
 [ 0.85488594 -0.5659634 ]
 [ 0.90676576 -0.53230476]
 [ 0.88996416 -0.5581274 ]
 [ 0.8901043  -0.54986024]
 [ 0.88130814 -0.5446439 ]
 [ 0.90812707 -0.5554699 ]
 [ 0.86252904 -0.5589392 ]
 [ 0.8999486  -0.535012  ]
 [ 0.86516935 -0.5685483 ]
 [ 0.8771035  -0.5504302 ]]


#####################################################


(array([[[ 5.3803831e-02,  1.0601948e+00, -1.6327662e+00, ...,
         -2.19

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


[[ 1.111169   -0.5850818 ]
 [ 1.1039554  -0.5691886 ]
 [ 1.0860136  -0.6091861 ]
 [ 1.1171491  -0.5837151 ]
 [ 1.1110878  -0.59052294]
 [ 1.0967689  -0.5715264 ]
 [ 1.036863   -0.59745383]
 [ 1.0985949  -0.5839584 ]
 [ 1.0787667  -0.5994135 ]
 [ 1.1153724  -0.59072757]
 [ 1.1047361  -0.5771488 ]
 [ 1.1194068  -0.595011  ]
 [ 1.1092222  -0.58134735]
 [ 1.076297   -0.57118887]
 [ 1.1003202  -0.5672349 ]
 [ 1.1041083  -0.5532637 ]
 [ 1.0746363  -0.56049955]
 [ 1.1087229  -0.5726277 ]
 [ 1.0975852  -0.5804913 ]
 [ 1.1000938  -0.59900934]
 [ 1.1111784  -0.58473384]
 [ 1.0899925  -0.5805702 ]
 [ 1.0798295  -0.6018157 ]
 [ 1.1221159  -0.5765649 ]
 [ 1.1000342  -0.5826001 ]
 [ 1.1075244  -0.5869407 ]
 [ 1.1033283  -0.583897  ]
 [ 1.1102834  -0.57625544]
 [ 1.0895159  -0.5951992 ]
 [ 1.1160427  -0.5721368 ]
 [ 1.0900447  -0.60910875]
 [ 1.100244   -0.5892098 ]]


#####################################################


(array([[[ 5.3863633e-02,  1.0582480e+00, -1.6327965e+00, ...,
         -2.22

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


In [ ]:
### long version of the same code ###
tokenizer = AutoTokenizer.from_pretrained("lanwuwei/BERTOverflow_stackoverflow_github", padding = True)
model = AutoModelForSequenceClassification.from_pretrained("lanwuwei/BERTOverflow_stackoverflow_github", num_labels = 2, problem_type = "single_label_classification")

data_files = {"train": "train.csv", "test": "test.csv"}
loaded_ds = load_dataset("csv", data_files = data_files)
print(f"shape of 'tokenized_train': {loaded_ds["train"].shape}")
print(f"shape of 'tokenized_test': {loaded_ds["test"].shape}")

small_train_val_ds = DatasetDict(
    train = loaded_ds["train"].shuffle(seed=42).select(range(128)),
    val = loaded_ds["train"].shuffle(seed=42).select(range(128, 160)),
)


small_tokenized_ds = small_train_val_ds.map(lambda example : tokenizer(example["text"], padding = True),
                                            batched = True,
                                            batch_size = 16,
                                            )
small_tokenized_ds = small_tokenized_ds.remove_columns(["text"])
small_tokenized_ds = small_tokenized_ds.rename_column("target", "labels")
small_tokenized_ds.set_format("torch")


In [125]:
len(small_tokenized_ds["train"]["input_ids"])
len(small_tokenized_ds["train"]["input_ids"])

len()

119